In [9]:
import pandas as pd
import graphlab as gb
import matplotlib.pyplot as plt
import seaborn as sb
get_ipython().magic(u'matplotlib inline')

In [2]:
df_original=pd.read_csv('lending-club-loan-data-original/loan.csv')
#df_refined=pd.read_csv('Project/lending-club-data.csv')

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [45]:
df_original['loan_status'].unique()

array(['Fully Paid', 'Charged Off', 'Current', 'Default',
       'Late (31-120 days)', 'In Grace Period', 'Late (16-30 days)',
       'Does not meet the credit policy. Status:Fully Paid',
       'Does not meet the credit policy. Status:Charged Off', 'Issued'], dtype=object)

In [56]:
#for prediction these matters
st=set(['Fully Paid','Default','Charged Off'])
def func(data):
    return (data in st)


In [57]:
l=df_original['loan_status'].apply(func)

In [65]:
df_original=df_original[l]

In [66]:
len(df_original)

254190

In [69]:
emptiness={}
for col in df_original.columns:
    empty=sum(df_original[col].isnull())
    emptiness[col]=empty

In [77]:
#sb.barplot(emptiness.values(),emptiness.keys())
full_filled=set()
nearly_unfilled=set()
others=set()
print "% emptiness of each column"
for items in emptiness.items():
    perc=items[1]*100.0/len(df_original)
    print items[0],"\t:",str(perc)+"%"    
    if perc<=15:
        full_filled.add(items[0])
    elif perc>=75:
        nearly_unfilled.add(items[0])
    else:
        others.add(items[0])

% emptiness of each column
mths_since_last_delinq 	: 55.6457767811%
il_util 	: 99.9504307801%
inq_last_6mths 	: 0.0%
tot_cur_bal 	: 25.0702230615%
grade 	: 0.0%
application_type 	: 0.0%
last_pymnt_amnt 	: 0.0%
open_acc_6m 	: 99.943349463%
annual_inc 	: 0.0%
total_acc 	: 0.0%
total_bal_il 	: 99.943349463%
out_prncp 	: 0.0%
emp_length 	: 0.0%
total_pymnt 	: 0.0%
out_prncp_inv 	: 0.0%
emp_title 	: 5.51044494276%
id 	: 0.0%
open_il_24m 	: 99.943349463%
pub_rec 	: 0.0%
verification_status_joint 	: 99.9996065935%
title 	: 0.00472087808332%
open_il_12m 	: 99.943349463%
funded_amnt_inv 	: 0.0%
initial_list_status 	: 0.0%
tot_coll_amt 	: 25.0702230615%
total_rec_prncp 	: 0.0%
next_pymnt_d 	: 99.520437468%
earliest_cr_line 	: 0.0%
total_rev_hi_lim 	: 25.0702230615%
zip_code 	: 0.0%
last_credit_pull_d 	: 0.00826153664582%
verification_status 	: 0.0%
total_pymnt_inv 	: 0.0%
open_acc 	: 0.0%
last_pymnt_d 	: 0.210865887722%
collections_12_mths_ex_med 	: 0.0220307643888%
dti 	: 0.0%
open_rv_12m 	: 99

In [78]:
print len(full_filled)
print "\n".join(sorted(full_filled))

49
acc_now_delinq
addr_state
annual_inc
application_type
collection_recovery_fee
collections_12_mths_ex_med
delinq_2yrs
dti
earliest_cr_line
emp_length
emp_title
funded_amnt
funded_amnt_inv
grade
home_ownership
id
initial_list_status
inq_last_6mths
installment
int_rate
issue_d
last_credit_pull_d
last_pymnt_amnt
last_pymnt_d
loan_amnt
loan_status
member_id
open_acc
out_prncp
out_prncp_inv
policy_code
pub_rec
purpose
pymnt_plan
recoveries
revol_bal
revol_util
sub_grade
term
title
total_acc
total_pymnt
total_pymnt_inv
total_rec_int
total_rec_late_fee
total_rec_prncp
url
verification_status
zip_code


In [79]:
print "\n".join(sorted(nearly_unfilled))

all_util
annual_inc_joint
dti_joint
il_util
inq_fi
inq_last_12m
max_bal_bc
mths_since_last_major_derog
mths_since_last_record
mths_since_rcnt_il
next_pymnt_d
open_acc_6m
open_il_12m
open_il_24m
open_il_6m
open_rv_12m
open_rv_24m
total_bal_il
total_cu_tl
verification_status_joint


In [80]:
print "\n".join(sorted(others))

desc
mths_since_last_delinq
tot_coll_amt
tot_cur_bal
total_rev_hi_lim


In [81]:
df_cleaned=df_original.drop(nearly_unfilled,axis=1)

In [82]:
df_cleaned.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim
0,1077501,1296599,5000.0,5000.0,4975.0,36 months,10.65,162.87,B,B2,...,Jan-2015,171.62,Jan-2016,0.0,1.0,INDIVIDUAL,0.0,NaN,NaN,NaN
1,1077430,1314167,2500.0,2500.0,2500.0,60 months,15.27,59.83,C,C4,...,Apr-2013,119.66,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,NaN,NaN,NaN
2,1077175,1313524,2400.0,2400.0,2400.0,36 months,15.96,84.33,C,C5,...,Jun-2014,649.91,Jan-2016,0.0,1.0,INDIVIDUAL,0.0,NaN,NaN,NaN
3,1076863,1277178,10000.0,10000.0,10000.0,36 months,13.49,339.31,C,C1,...,Jan-2015,357.48,Jan-2015,0.0,1.0,INDIVIDUAL,0.0,NaN,NaN,NaN
5,1075269,1311441,5000.0,5000.0,5000.0,36 months,7.90,156.46,A,A4,...,Jan-2015,161.03,Sep-2015,0.0,1.0,INDIVIDUAL,0.0,NaN,NaN,NaN


In [84]:
len(df_cleaned.columns)

54

In [96]:
numerical_cols=df_cleaned.describe().columns

In [95]:
len(numerical_cols)

33

In [98]:
df_cleaned[numerical_cols].head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,...,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,policy_code,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim
0,1077501,1296599,5000.0,5000.0,4975.0,10.65,162.87,24000.0,27.65,0.0,...,0.00,0.00,0.00,171.62,0.0,1.0,0.0,NaN,NaN,NaN
1,1077430,1314167,2500.0,2500.0,2500.0,15.27,59.83,30000.0,1.00,0.0,...,0.00,117.08,1.11,119.66,0.0,1.0,0.0,NaN,NaN,NaN
2,1077175,1313524,2400.0,2400.0,2400.0,15.96,84.33,12252.0,8.72,0.0,...,0.00,0.00,0.00,649.91,0.0,1.0,0.0,NaN,NaN,NaN
3,1076863,1277178,10000.0,10000.0,10000.0,13.49,339.31,49200.0,20.00,0.0,...,16.97,0.00,0.00,357.48,0.0,1.0,0.0,NaN,NaN,NaN
5,1075269,1311441,5000.0,5000.0,5000.0,7.90,156.46,36000.0,11.20,0.0,...,0.00,0.00,0.00,161.03,0.0,1.0,0.0,NaN,NaN,NaN


In [99]:
for col in numerical_cols:
    empty=sum(df_cleaned[col].isnull())
    if empty>0:
        print col

mths_since_last_delinq
revol_util
collections_12_mths_ex_med
tot_coll_amt
tot_cur_bal
total_rev_hi_lim


In [100]:
df_cleaned['mths_since_last_delinq']=df_cleaned["mths_since_last_delinq"].fillna(0)

In [117]:
df_cleaned["revol_util"]=df_cleaned["revol_util"].fillna(df_cleaned['revol_util'].value_counts().index[0])

In [103]:
df_cleaned["collections_12_mths_ex_med"]=df_cleaned["collections_12_mths_ex_med"].fillna(df_cleaned["collections_12_mths_ex_med"].median())

In [108]:
sum(df_cleaned[df_cleaned['tot_coll_amt'].isnull()]['loan_status']=='Default')

18

In [109]:
df_cleaned["tot_coll_amt"]=df_cleaned["tot_coll_amt"].fillna(0)

In [112]:
print df_cleaned[df_cleaned['tot_cur_bal'].isnull()]['loan_status'].unique()
print sum(df_cleaned[df_cleaned['tot_cur_bal'].isnull()]['loan_status']=='Default')
print sum(df_cleaned[df_cleaned['tot_cur_bal'].isnull()]['loan_status']=='Charged Off')
print sum(df_cleaned[df_cleaned['tot_cur_bal'].isnull()]['loan_status']=='Fully Paid')

['Fully Paid' 'Charged Off' 'Default']
18
10019
53689


In [113]:
df_cleaned["tot_cur_bal"]=df_cleaned["tot_cur_bal"].fillna(0)

In [118]:
df_cleaned["total_rev_hi_lim"]=df_cleaned["total_rev_hi_lim"].fillna(df_cleaned['total_rev_hi_lim'].value_counts().index[0])

In [119]:
for col in numerical_cols:
    empty=sum(df_cleaned[col].isnull())
    if empty>0:
        print col

In [123]:
categorical_cols=list(set(df_cleaned.columns)-set(numerical_cols))

In [124]:
df_cleaned[categorical_cols].head()

,last_credit_pull_d,verification_status,term,issue_d,title,grade,application_type,initial_list_status,loan_status,sub_grade,...,last_pymnt_d,earliest_cr_line,pymnt_plan,purpose,emp_length,url,addr_state,desc,emp_title,zip_code
0,Jan-2016,Verified,36 months,Dec-2011,Computer,B,INDIVIDUAL,f,Fully Paid,B2,...,Jan-2015,Jan-1985,n,credit_card,10+ years,https://www.lendingclub.com/browse/loanDetail....,AZ,Borrower added on 12/22/11 > I need to upgra...,NaN,860xx
1,Sep-2013,Source Verified,60 months,Dec-2011,bike,C,INDIVIDUAL,f,Charged Off,C4,...,Apr-2013,Apr-1999,n,car,< 1 year,https://www.lendingclub.com/browse/loanDetail....,GA,Borrower added on 12/22/11 > I plan to use t...,Ryder,309xx
2,Jan-2016,Not Verified,36 months,Dec-2011,real estate business,C,INDIVIDUAL,f,Fully Paid,C5,...,Jun-2014,Nov-2001,n,small_business,10+ years,https://www.lendingclub.com/browse/loanDetail....,IL,NaN,NaN,606xx
3,Jan-2015,Source Verified,36 months,Dec-2011,personel,C,INDIVIDUAL,f,Fully Paid,C1,...,Jan-2015,Feb-1996,n,other,10+ years,https://www.lendingclub.com/browse/loanDetail....,CA,Borrower added on 12/21/11 > to pay for prop...,AIR RESOURCES BOARD,917xx
5,Sep-2015,Source Verified,36 months,Dec-2011,My wedding loan I promise to pay back,A,INDIVIDUAL,f,Fully Paid,A4,...,Jan-2015,Nov-2004,n,wedding,3 years,https://www.lendingclub.com/browse/loanDetail....,AZ,NaN,Veolia Transportaton,852xx


In [127]:
for col in categorical_cols:
    empty=sum(df_cleaned[col].isnull())
    if empty>0:
        print col,empty

last_credit_pull_d 21
title 12
last_pymnt_d 536
desc 165744
emp_title 14007


In [128]:
df_cleaned["emp_title"]=df_cleaned["emp_title"].fillna(df_cleaned['emp_title'].value_counts().index[0])

In [129]:
df_cleaned["title"]=df_cleaned["title"].fillna('Others')

In [ ]:
to_be_removed=['url','desc']

In [130]:
df_cleaned=df_cleaned.drop(to_be_removed,axis=1)

In [134]:
def fun(data):
    if data in ['Default','Charged Off']:
        return 0
    else:
        return 1   

In [135]:
df_cleaned['labels']=df_cleaned['loan_status'].apply(fun)

In [137]:
df_cleaned[['labels','loan_status']].head(10)

,labels,loan_status
0,1,Fully Paid
1,0,Charged Off
2,1,Fully Paid
3,1,Fully Paid
5,1,Fully Paid
7,1,Fully Paid
8,0,Charged Off
9,0,Charged Off
10,1,Fully Paid
11,1,Fully Paid


In [138]:
df_cleaned.to_csv('loan_cleaned.csv',index=False)

In [145]:
(df_cleaned.columns)

Index([u'id', u'member_id', u'loan_amnt', u'funded_amnt', u'funded_amnt_inv',
       u'term', u'int_rate', u'installment', u'grade', u'sub_grade',
       u'emp_title', u'emp_length', u'home_ownership', u'annual_inc',
       u'verification_status', u'issue_d', u'loan_status', u'pymnt_plan',
       u'purpose', u'title', u'zip_code', u'addr_state', u'dti',
       u'delinq_2yrs', u'earliest_cr_line', u'inq_last_6mths',
       u'mths_since_last_delinq', u'open_acc', u'pub_rec', u'revol_bal',
       u'revol_util', u'total_acc', u'initial_list_status', u'out_prncp',
       u'out_prncp_inv', u'total_pymnt', u'total_pymnt_inv',
       u'total_rec_prncp', u'total_rec_int', u'total_rec_late_fee',
       u'recoveries', u'collection_recovery_fee', u'last_pymnt_d',
       u'last_pymnt_amnt', u'last_credit_pull_d',
       u'collections_12_mths_ex_med', u'policy_code', u'application_type',
       u'acc_now_delinq', u'tot_coll_amt', u'tot_cur_bal', u'total_rev_hi_lim',
       u'labels'],
      dtype='o

In [132]:
len(df_cleaned)

254190

In [ ]:
y = df_cleaned.pop('loan_status')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42)

In [133]:
from sklearn.cross_validation import train_test_split

In [30]:
good_features=['grade','sub_grade','short_emp','emp_length_num','home_ownership','dti','purpose','term','last_delinq_none','last_major_derog_none','revol_util','total_rec_late_fee']
good_features.sort()
print "\n".join(good_features)

dti
emp_length_num
grade
home_ownership
last_delinq_none
last_major_derog_none
purpose
revol_util
short_emp
sub_grade
term
total_rec_late_fee
